In [1]:
import numpy as np
import matplotlib.pyplot as plt

# load train/test data from npy
data = np.load('train.npy')
labels = np.load('train_labels.npy')

# split into train and test
trainlen = int(len(labels) * 0.7)
train_data = data[:trainlen]
train_labels = labels[:trainlen]

test_data = data[trainlen:]
test_labels = labels[trainlen:]

In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y_ = tf.placeholder(tf.float32, shape=[None, 26])

In [4]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [5]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [6]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.elu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.elu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [7]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.elu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [8]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [10]:
W_fc2 = weight_variable([1024, 525])
b_fc2 = bias_variable([525])
y_conv = tf.nn.elu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
yc1_drop = tf.nn.dropout(y_conv, keep_prob)

W_fc3 = weight_variable([525, 26])
b_fc3 = bias_variable([26])
y_conv2 = tf.matmul(yc1_drop, W_fc3) + b_fc3

In [14]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv2))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv2, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
index = 0
def next_batch(batch_size):
    global index
    result = (train_data[index:(index + batch_size)], train_labels[index:(index + batch_size)])
    index += batch_size
    index %= len(train_data)
    return result

In [16]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(100000):
    batch = next_batch(64)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: test_data, y_: test_labels, keep_prob: 1.0}))
  saver = tf.train.Saver()
  saver.save(sess, 'my_test_model',global_step=1000)

step 0, training accuracy 0.0625


KeyboardInterrupt: 